In [ ]:
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, NamedStyle
from openpyxl.utils import get_column_letter

export_path = r"C:\Users\User\Downloads\COB DATA\2025\10.October\Partial\Exports"
import_path = r"C:\Users\User\Downloads\COB DATA\2025\10.October\Partial\Data"
os.makedirs(export_path, exist_ok=True)

category_df = pd.read_excel(
    r"C:\Users\User\OneDrive\Kimberlin Enterprises\REPORTS\References\CATEGORY.xlsx")

field_supervisors_df = pd.read_excel(
    r"C:\Users\User\OneDrive\Kimberlin Enterprises\REPORTS\References\Field Supervisors.xlsx")

week_df = pd.read_excel(
    r"C:\Users\User\OneDrive\Kimberlin Enterprises\REPORTS\References\WEEK.xlsx")


In [ ]:
#CUSTOMER RETURNS Arrangement

im_filename_ret = "DMS-Customer Returns-on.xlsx"
full_im_path_ret = os.path.join(import_path, im_filename_ret)

df2 = pd.read_excel(full_im_path_ret)

dfc2 = df2[['Customer Return Date', 'Sold To Customer Number', 'Sold To Customer Name', 
'Product Code', 'Product Description', 'Facility Name', 'Estimated Product Return Amount']]
dfc2['Estimated Product Return Amount'].fillna(0, inplace=True)
dfc2.insert(5,'Total Item amount with Tax and Discount', 0)
dfc2.insert(8,'with vat', 0)
dfc2['with vat'] = dfc2['Estimated Product Return Amount'] * 1.12

cust_ret = dfc2.pivot_table(index=['Customer Return Date', 'Sold To Customer Number', 'Sold To Customer Name', 
'Product Code', 'Product Description', 'Total Item amount with Tax and Discount'], columns='Facility Name', values='with vat', aggfunc=sum)

ret_f = cust_ret.reset_index()

ret_f['Product Code'] = ret_f['Product Code'].astype(str)
ret_f['BO'] = ret_f['BO'].fillna(0)
ret_f['FG'] = ret_f['FG'].fillna(0)

In [ ]:
#INVOICE ON Arrangement

im_filename_inv = "DMS-Invoice-on.xlsx"
full_im_path_inv = os.path.join(import_path, im_filename_inv)

df1 = pd.read_excel(full_im_path_inv)

dfc = df1[['Invoice Date', 'Sold To Customer Number',
'Sold To Customer Name', 'Product Code', 'Product/Item Description',
'Total Item amount with Tax and Discount', 'Invoice Item Type']]
inv = dfc[dfc['Invoice Item Type'] != 'ITM_SALES_TAX']

inv_f = inv
inv_f.insert(6,'BO',0)
inv_f.insert(7,'FG',0)
inv_f['Total Item amount with Tax and Discount'] = inv_f['Total Item amount with Tax and Discount'].fillna(0)
inv_f = inv_f.drop('Invoice Item Type', axis=1)

In [ ]:
#Customer Master List

im_filename_cust = "DMS-Customer-on.xlsx"
full_im_path_cust = os.path.join(import_path, im_filename_cust)

cust_df = pd.read_excel(full_im_path_cust)

cust_df = cust_df[[
    'NEXT_UP_NUMBER',
    'PARTY_CLASSIFICATION_DESCRIPTION',
    'KEY_ACCOUNT',
    'SALES_REP_ID',
    'SALES_REP_NAME',
    'BRANCH_NAME',
    'GEO_LOCATION_HIERARCHYDESCRIPTION',
    'CITY',
    'STATE_PROVINCE',
    'CHANNEL'
]]

In [ ]:
#Price List M0

im_filename_pl = "DMS-Price-on.xlsx"
full_im_path_pl = os.path.join(import_path, im_filename_pl)

pl_df = pd.read_excel(full_im_path_pl)

pl_df2 = pl_df

pl_df2 = pl_df2[['product_code', 'product_description', 'uom_description', 'selling_price', 'cust_class', 'cust_channel']]
pl_df2.insert(4,'with vat', 0)
pl_df2['with vat'] = pl_df2['selling_price'] * 1.12

pl_df_m0 = pl_df2

pl_df_m0 = pl_df_m0[pl_df_m0['cust_class'] != 'BEV Dealer']
pl_df_m0 = pl_df_m0[pl_df_m0['cust_channel'] != 'VAN(EXTRUCK)']

pl_df_m0_f = pl_df_m0.pivot_table(index=['product_code', 'product_description',], columns='uom_description', values='with vat', aggfunc=sum)

pl_df_m0_f_1 = pl_df_m0_f.reset_index()

pl_df_m0_f_1['Case'] = pl_df_m0_f_1['Case'].fillna(0)
pl_df_m0_f_1['Piece'] = pl_df_m0_f_1['Piece'].fillna(0)
pl_df_m0_f_1['Subcase'] = pl_df_m0_f_1['Subcase'].fillna(0)

pl_m0_final = pl_df_m0_f_1[['product_code', 'product_description', 'Case', 'Subcase', 'Piece']]

ex_filename_pl_m0 = "M0 Pricelist.xlsx"
full_path_pl_m0 = os.path.join(export_path, ex_filename_pl_m0)

export_pl_m0 = pl_m0_final
export_pl_m0.to_excel(full_path_pl_m0, index=False)

In [ ]:
#COMBINE INVOICED AND RETURNS to NET INVOICED

inv_f.rename(columns={
'Invoice Date': 'DATE',
'Sold To Customer Number': 'ACCOUNT CODE',
'Sold To Customer Name': 'ACCOUNT NAME',
'Product Code': 'SKU CODE',
'Product/Item Description': 'SKU NAME',
'Total Item amount with Tax and Discount': 'SERVED INVOICE',
'BO': 'BAD RETURNS',
'FG': 'GOOD RETURNS'
}, inplace=True)

ret_f.rename(columns={
'Customer Return Date': 'DATE',
'Sold To Customer Number': 'ACCOUNT CODE', 
'Sold To Customer Name': 'ACCOUNT NAME',
'Product Code': 'SKU CODE', 
'Product Description': 'SKU NAME',
'Total Item amount with Tax and Discount': 'SERVED INVOICE',
'BO': 'BAD RETURNS',
'FG': 'GOOD RETURNS'
}, inplace=True)

net_inv = pd.concat([inv_f, ret_f], axis=0)

ex_filename_net_inv = "Net_Invoiced.xlsx"
full_path_net_inv = os.path.join(export_path, ex_filename_net_inv)

# Customer Details Arrangements

cml_df = cust_df.rename(columns={
    'NEXT_UP_NUMBER': 'ACCOUNT CODE'
})

net_inv_f = net_inv.merge(cml_df, on='ACCOUNT CODE', how='left')

# export_net_inv = net_inv_f
# export_net_inv = export_net_inv.sort_values(by=['DATE', 'ACCOUNT CODE'])
# export_net_inv.to_excel(full_path_net_inv, index=False)

category_df = category_df.drop(['SKU NAME'], axis=1)
net_inv_f_l1 = net_inv_f.merge(category_df,  on='SKU CODE', how='left')

net_inv_f_l2 = net_inv_f_l1.merge(field_supervisors_df, on='SALES_REP_ID', how='left')

net_inv_f_l3 = net_inv_f_l2.merge(week_df, on='DATE', how='inner')

# Get Volume - Price Arrangements

pl_m0_final_reference = pl_m0_final[['product_code', 'Case']]

pl_m0_final_reference = pl_m0_final_reference.rename(columns={
    'product_code': 'SKU CODE',
    'Case': 'SKU PRICE REFERENCE'
})

net_inv_f_l4 = net_inv_f_l3.merge(pl_m0_final_reference, on='SKU CODE', how='left')

net_inv_f_l4.insert(5,'VALUE', 0)
net_inv_f_l4['VALUE'] = net_inv_f_l4['SERVED INVOICE'] - net_inv_f_l4['BAD RETURNS'] - net_inv_f_l4['GOOD RETURNS']

# ensure numeric and avoid divide-by-zero when computing VOLUME
net_inv_f_l4['SKU PRICE REFERENCE'] = pd.to_numeric(net_inv_f_l4['SKU PRICE REFERENCE'], errors='coerce').fillna(0)
net_inv_f_l4['VOLUME'] = 0
mask = net_inv_f_l4['SKU PRICE REFERENCE'] != 0
net_inv_f_l4.loc[mask, 'VOLUME'] = net_inv_f_l4.loc[mask, 'VALUE'] / net_inv_f_l4.loc[mask, 'SKU PRICE REFERENCE']

# Post-Net Invoiced

export_net_inv = net_inv_f_l4
export_net_inv['RD NAME'] = 'Kimberlin'

export_net_inv = export_net_inv.sort_values(by=['DATE', 'ACCOUNT CODE'])

export_net_inv.rename(columns={
    'RD NAME': 'RD Name',
    'DATE': 'Date',
    'WEEK': 'Week',
    'BRANCH_NAME': 'Branch Name',
    'SALES_REP_ID': 'Employee Code',
    'SALES_REP_NAME': 'Employee Name',
    'KEY_ACCOUNT': 'Channel',
    'ACCOUNT CODE': 'Sold To Customer number',
    'ACCOUNT NAME': 'Sold To Customer Name',
    'CATEGORY': 'Category',
    'SKU CODE': 'Product Code',
    'SKU NAME': 'Product Description',
    'VOLUME': 'Volume',
    'VALUE': 'Net Value',
    'GOOD RETURNS': 'Good Stock Returns',
    'BAD RETURNS': 'Bad Stock Returns',
    'PARTY_CLASSIFICATION_DESCRIPTION': 'Channel_Classification',
    'GEO_LOCATION_HIERARCHYDESCRIPTION': 'Brgy',
    'CITY': 'Town',
    'STATE_PROVINCE': 'Province',
    'FS': 'FS',
    'CHANNEL': 'RTM Model'
}, inplace=True)

export_net_inv_final = export_net_inv[[
    'RD Name',
    'Date',
    'Week',
    'Branch Name',
    'Employee Code',
    'Employee Name',
    'Channel',
    'Sold To Customer number',
    'Sold To Customer Name',
    'Category',
    'Product Code',
    'Product Description',
    'Volume',
    'Net Value',
    'Good Stock Returns',
    'Bad Stock Returns',
    'Channel_Classification',
    'Brgy',
    'Town',
    'Province',
    'FS',
    'RTM Model',
]]

# Net Invoiced Finalization

# Step 1: Export to Excel with a blank row at the top

temp_output = 'temp_file_output.xlsx'
temp_file = os.path.join(import_path, temp_output)

with pd.ExcelWriter(temp_file, engine='openpyxl') as writer:
    # Write to row 2 (startrow=1) to leave row 1 blank
    export_net_inv_final.to_excel(writer, index=False, startrow=1)

# Step 2: Load workbook and apply formatting
wb = load_workbook(temp_file)
ws = wb.active

# Step 3: Style header (row 2)
header_fill = PatternFill(start_color='000000', end_color='000000', fill_type='solid')
header_font = Font(bold=True, color='FFFFFF')
header_alignment = Alignment(horizontal='center')

for cell in ws[2]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_alignment

# Step 4: Format numeric columns with comma style
accounting_style = NamedStyle(name="accounting_style", number_format='_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)')
for col in range(13, 17):  # Assuming numeric columns start from column 2
    for row in range(3, ws.max_row + 1):  # Data starts from row 3
        cell = ws.cell(row=row, column=col)
        if isinstance(cell.value, (int, float)):
            cell.style = accounting_style

sum_row = ws.max_row + 1
start_range = 3
end_range = ws.max_row

for col in range(13, 17):
    col_letter = get_column_letter(col)
    sum_cell = ws.cell(row=sum_row, column=col)
    sum_cell.value = f"=SUM({col_letter}{start_range}:{col_letter}{end_range})"
    sum_cell.font = Font(bold=True, color="FF0000")
    sum_cell.number_format = '_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)'

# List the column letters or indexes you want to auto-fit
columns_to_adjust = ['M', 'N', 'O', 'P']  # or use [2, 4] for indexes

for col_id in columns_to_adjust:
    # Convert index to letter if needed
    col_letter = get_column_letter(col_id) if isinstance(col_id, int) else col_id
    max_length = 0

    for cell in ws[col_letter]:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))

    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding


# Save changes
Net_Invoiced = "Net_Invoiced.xlsx"
Net_Invoiced_path = os.path.join(export_path, Net_Invoiced)

ws.sheet_view.showGridLines = False
wb.save(Net_Invoiced_path)